## Usage

<div style="background: #efffed;
            border: 1px solid grey;
            margin: 8px 0 8px 0;
            text-align: center;
            padding: 8px; ">
    <i class="fa-play fa" 
       style="font-size: 40px;
              line-height: 40px;
              margin: 8px;
              color: #444;">
    </i>
    <center>
    To run a selected code cell, hit <pre style="background: #efffed">Shift + Enter</pre>
    </center>
</div>

# Introduction

## Technical prerequisites
* none

## Conceptual prerequisites
* familiarity with mineral stability diagrams
* using log rules to linearize equilibrium equations
* plotting in log space

## Learning goals
* plot equilibrium lines on a mineral stability diagram using R
* improve visualization of reaction paths
* connect groundwater chemistry to mineral assemblages

## Background information
A mineral stability diagram shows the boundaries between the thermodynamic stability regions of different minerals (i.e., where the mineral begins to precipitate) as a function of two or more master variables, often the activities or activity ratios of the major controlling species present (Kinniburgh and Cooper, 2004 https://pubs.acs.org/doi/pdf/10.1021/es034927l)

One application of mineral stability diagrams is to trace how _primary minerals_, which are often igneous or metamorphic, react _(weather)_ into _secondary mineral_ products when primary minerals contact water with which they are not in thermodynamic equilibrium.

In this Jupyter notebook (run on an R kernel), we derive a mineral stability diagram, then calculate and visualize changes in water chemistry and secondary mineral assemblages as the common igneous primary mineral microcline (a form of potassium feldspar) reacts with a typical, mildly acidic soil water. Chemical weathering is a ubiquitous process on Earth's surface that is of fundamental importance due to its impact on the aqueous chemistry and mineralogy, as we will observe.

By the way, Jupyter notebooks (as well as R markdown documents) weave together narrative text and code to produce elegantly formatted output. Plus, they are fully reproducible. All the cool kids are using them!

Before diving into the chemistry, let's visualize the mineral stability diagram for the $\text{K}_2\text{O} - \text{Al}_2\text{O}_3 - \text{SiO}_2 - \text{H}_2\text{O}$ system.

First, we need to load some packages. Packages contain functions that other smarty-pants wizards wrote. That's one great thing about R: no need to re-invent the wheel. Run this code chunk by clicking into it and pressing Shift + Enter.

# Setup

In [ ]:
# load libraries
suppressMessages({
    library(tidyverse) # dplyr, tidyr, ggplot
    library(plotly) # interactive plots
    library(CHNOSZ) # stability diagrams
    library(latex2exp) # writing compounds and greek letters in plots
})
data(thermo) # load thermodynamics constants
source(file.path("scripts", "functions.R")) # load custom functions
options(repr.plot.width = 6, repr.plot.height = 4) # default plot size


# Mineral stability diagram for the $\text{K}_2\text{O} - \text{Al}_2\text{O}_3 - \text{SiO}_2 - \text{H}_2\text{O}$ system
The R package 'CHNOSZ' contains a speciation function, which calculates the distribution of chemical species based on their thermodynamic characteristics, which are stored in a large database contained in the package. To generate the mineral stability diagram for the $\text{K}_2\text{O} - \text{Al}_2\text{O}_3 - \text{SiO}_2 - \text{H}_2\text{O}$ system at T = 25˚C and P = 1 bar using the CHNOSZ package, run the chunk below.



In [ ]:
## K2O-Al2O3-SiO2-H2O, 25 degree C, 1 bar
## Steinmann et al., 1994 (http://ccm.geoscienceworld.org/content/42/2/197)
## Garrels and Christ, p. 361 (http://www.worldcat.org/oclc/517586)
## https://wwwbrr.cr.usgs.gov/projects/GWC_coupled/phreeqc/html/final-75.html
resolution <- 200
basis <- basis(c("Al+3", "pseudo-H4SiO4", "K+", "H2O", "H+", "O2"))
species <- species(c("gibbsite", "muscovite", "kaolinite", "K-feldspar"))
a <- affinity(H4SiO4 = c(-6, -2, resolution), `K+` = c(-3, 6, resolution))

# process
plot <- a %>%
  # fetch tidy data frame
  get_diagram_data() %>%
  # plot as ggplot
  plot_diagram_data() +
  # add labels
  labs(
    x = latex2exp::TeX("$\\log\\lbrack H_4SiO_4\\rbrack$$"),
    y = latex2exp::TeX("$\\log\\frac{\\lbrack K^+ \\rbrack}{\\lbrack H^+ \\rbrack}$"),
    fill = "")

# print plot
print(plot)

Wow, we just pressed 2 buttons and made a lot of science happen! But what is the above diagram telling us? The x-axis of the diagram is $\text{log}[\text{H}_{4}\text{SiO}_{4}]$, which is silicic acid. The Y-axis is $\text{log}\frac {[\text{K}^{+}]}{[\text{H}^{+}]}$. These species were chosen because they control the stability of the minerals in this sytem, but different species could be plotted depending on the system of interest. Soon, we will see why the log concentrations are plotted. This diagram shows the stable mineral at a wide range of water chemistry conditions.

## Question 1

What is the stable mineral under the following conditions: $\text{log}[\text{H}_{4}\text{SiO}_{4}]=-4$, $\text{log}\frac {[\text{K}^{+}]}{[\text{H}^{+}]}=0$, T = 25˚C and P = 1 bar?

Now that you can visualize and interpret a mineral stability diagram, let's investigate how it is actually created.

You should have already derived in class all the equilibria that are relevant to the system we've been discussing. For your reference, the equations and equilibrium constants, $K$, at 25˚C and 1 bar are provided below.

**kaolinite - gibbsite**
$\text{Al}_3\text{Si}_2\text{O}_5\text{(OH)}_4 \text{(s)} + 5\text{H}_2\text{O} \text{(l)} \leftrightharpoons 2\text{Al}\text{(OH)}_3 \text{(s)} + 2\text{H}_4\text{SiO}_4 \text{(aq)}, \space  K =10^{-8.491} \space \text{(reaction 1)}$

**muscovite - gibbsite**
$\text{KAl}_3\text{Si}_3\text{O}_{10}\text{(OH)}_2 \text{(s)} + 9\text{H}_2\text{O} \text{(l)}+ \text{H}^+ \text{(aq)} \leftrightharpoons 3\text{Al}\text{(OH)}_3 \text{(s)} + \text{K}^+ \text{(aq)}+ 3\text{H}_4\text{SiO}_4 \text{(aq)}, K = \text{10}^{-9.366} \space \text{(reaction 2)}$

**muscovite - kaolinite**
$2\text{KAl}_3\text{Si}_3\text{O}_{10}\text{(OH)}_2 \text{(s)} + 3\text{H}_2\text{O} \text{(l)}+ 2\text{H}^+ \text{(aq)} \leftrightharpoons 3\text{Al}_2\text{Si}_2\text{O}_5\text{(OH)}_4 \text{(s)} + 2 \text {K}^+ \text{(aq)}, \space K = 10^{6.741} \space \text{(reaction 3)}$

**microcline - muscovite**
$3\text{KAlSi}_3\text{O}_8 \text{(s)}+ 2\text{H}^+ \text{(aq)} + 12\text{H}_2\text{O} \text{(l)} \leftrightharpoons \text{KAl}_3\text{Si}_3\text{O}_{10}\text{(OH)}_2 \text{(s)} + 2 \text {K}^+ \text{(aq)} + 6\text{H}_{4}\text{SiO}_{4} \text{(aq)},\space K = 10^{-14.412} \space \text{(reaction 4)}$

**microcline - kaolinite**
$2\text{KAlSi}_3\text{O}_8 \text{(s)} + 2\text{H}^+ \text{(aq)}+ 9\text{H}_2\text{O} \text{(l)} \leftrightharpoons  \text{Al}_2\text{Si}_2\text{O}_5\text{(OH)}_4 \text{(s)}+ 2\text{K}^+ \text{(aq)} + 4\text{H}_{4}\text{SiO}_{4} \text{(aq)} \space K =\text {10}^{-7.361} \space \text{(reaction 5)}$

We can relate these equations to their equilibrium constants through the law of mass action, i.e. for Kaolinite - Gibbsite (reaction 1):

$\left[\text{H}_4\text{SiO}_4\right]^2=10^{-8.491}$ (recall that solids and water have activity of 1 and thus cancel)

How could we visualize this? Note that the right side of the equation contains $10^{-8.491}$. Thus if we apply our exponent and log rules, and take the $\log_{10}$ of both sides, we can rearrange the above equation to:

$\log\left[\text{H}_4\text{SiO}_4\right]=-4.2455$

Let's run the following chunk to assign our solution to a numeric value using "<-" so we can use it later.

In [ ]:
# solve for Y coordinate of the two intersecting equilibrium lines 
# and assign it to a numeric variable
gibb_intersect_H4SiO4 <- -4.2455 

# print the value
print(gibb_intersect_H4SiO4)

Now, we can plot the above equation as a vertical line with $\log\left[\text{H}_4\text{SiO}_4\right]$ on the X axis. This line indicates water chemistries at which gibbsite and kaolinite can coexist at equilibrium. Run the code chunk below to see this line plotted. Note: we're using the ggplot2 package for plotting here, and using the geom_vline() function to plot a vertical line. You can see in the code that we've written in the X intercept of -4.2455. We'll be using comments (un-executed code) with a #hashtag symbol to guide you through the relevant pieces of code.



In [ ]:
# setup a basic plot
base_plot <- 
    ggplot()+
    # scales
    scale_x_continuous(limits = c(-6, -2), expand = c(0, 0))+
    scale_y_continuous(limits = c(-3, 6), expand = c(0, 0))+
    # labels
    labs(
        x = TeX("$\\log\\lbrack H_4SiO_4\\rbrack$"),
        y = TeX("$\\log\\frac{\\lbrack K^+ \\rbrack}{\\lbrack H^+ \\rbrack}$")
    ) + 
    # style
    theme_bw()+
    theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank())+
    theme(axis.title.y = element_text(angle = 0, vjust = .5, size = 11))

# add the first line to the base plot
plot_stability_segment <- 
    base_plot + 
    # add vertical line for gibbsite - kaolinite equilibrium
    geom_vline(xintercept = gibb_intersect_H4SiO4) 

# print plot
print(plot_stability_segment)

As we can now see, plotting in log coordinates has allowed us to visualize equilibrium equations as straight lines. We are well on our way to completing a mineral stability diagram!

Now let's see what happens if we try to add another equilibrium line to the diagram. This time, we'll use the Muscovite - Gibbsite equilibrium (reaction 2 from above)

$\text{KAl}_3\text{Si}_3\text{O}_{10}\text{(OH)}_2 + 9\text{H}_2\text{O} + \text{H}^+ \leftrightharpoons 3\text{Al}\text{(OH)}_3  + \text{K}^+ + 3\text{H}_4\text{SiO}_4, K = \text{10}^{-9.366}$

Relating concentrations to the equilibrium constant with the Law of Mass Action:
$\frac{\left[\text{H}_4\text{SiO}_4\right]^3\left[\text{K}^+\right]}{\left[\text{H}^+\right]}=10^{-9.366}$

Taking log of both sides:
$3\log\left[\text{H}_4\text{SiO}_4\right]+\log\frac{\left[\text{K}^+\right]}{\left[\text{H}^+\right]}=-9.366$

Rearranging:
$\log\frac{\left[\text{K}^+\right]}{\left[\text{H}^+\right]}=-3\log\left[\text{H}_4\text{SiO}_4\right]-9.366$

The above expression is the equation for a straight line in the form Y=mX+b on a plot of $\log\frac{\left[\text{K}^+\right]}{\left[\text{H}^+\right]}$ vs. $\log\left[\text{H}_4\text{SiO}_4\right]$

We can add this line to our mineral stability plot using the geom_abline() function, which takes the slope and Y-intercept of a line as arguments. See and run the code chunk below.


In [ ]:
plot_stability_segment2 <- 
    # start with the previous plot
    plot_stability_segment + 
    # add straight line in slope-intercept form for muscovite gibbsite equilibrium
    geom_abline(intercept = -9.366, slope = -3)

# print plot
print(plot_stability_segment2)


Cool, it's starting to look like a mineral stability diagram, but as you can see, the lines intersect. How do we deal with this? First, let's annotate the text "gibbsite" in its stability field, just like in the plot we made with the CHNOSZ package. We can do this using the annotate() function. We just tell it what we want to annotate (text), the X and Y coordinates, of our annotation, as well as what should be on the label (gibbsite), and the size of the letters (4). See and run the chunk.



In [ ]:
plot_stability_segment3 <- 
    # start with the previous plot
    plot_stability_segment2 + 
    # add gibbsite text label
    annotate("text", y = 4, x = -5.4, label = "gibbsite", size = 4)

# print plot
print(plot_stability_segment3)

## Question 2
How you could infer which of the areas on the above plot is the stability field for gibbsite using Le Châtelier's Principle? Explain.


Since gibbsite is only stable in one region on this plot, we must remove the excess lines, which are meaningless. To do this, we must first solve for the point at which these two lines intersect. The X coordinate of the point of intersection will be -4.2455, which we take for the equation of a vertical line that we derived for the Kaolinite - Gibbsite equilibrium. We can then simply plug in that X value to the equilibrium line we derived for the Muscovite - Gibbsite equilibrium. Run the chunk below to plug calculate the Y coordinate of the point of intersection.

In [ ]:
# solve for Y coordinate of the two intersecting equilibrium lines
# and assign it to a numeric value
gibb_intersect_KH <- -3 * -4.2455 - 9.366 

# print the value
print(gibb_intersect_KH)


Now, we can use the geom_segment() function to plot two line segments that define the borders of the gibbsite stability field. The geom_segment() function takes two XY coordinate points as arguments, and draws a line through them. In our case, the lines hit the edges of the region we have chosen to display on our plot. For the vertical line, it is simple to infer where it hits the edge, but for the sloped line, we must solve for it.





In [ ]:
# solve for X coordinate where gibbsite-muscovite equilibrium line exits the plot
# and assign it to a numeric value
gibb_musc_edge_H4SiO4 <- -(6 + 9.366) / 3 

# print that value
print(gibb_musc_edge_H4SiO4)

In [ ]:
plot_stability_segment4 <- 
    # start back with the base plot
    base_plot + 
    # gibbsite - kaolinite equilibrium line
    geom_segment(aes(x = gibb_intersect_H4SiO4, y = gibb_intersect_KH, xend = gibb_intersect_H4SiO4, yend = -3)) +
    # gibbsite - muscovite equilibrium line
    geom_segment(aes(x = gibb_intersect_H4SiO4, y = gibb_intersect_KH, xend = gibb_musc_edge_H4SiO4, yend = 6)) + 
    # gibbsite label
    annotate("text", y = 4, x=-5.4, label = "gibbsite", size = 4)

# print plot
print(plot_stability_segment4)

## Question 3

OK now it's your turn. Solve for the equilibrium lines for Muscovite - Kaolinite and Microcline - Muscovite, and then input the endpoints of segments in the chunk below to plot them. Please do your calculations on a separate sheet of paper for your instructors' review.  *In case you need a refresher, there is a Exponent and Logarithm Rule Cheat Sheet in the repository for your reference.*

**muscovite - kaolinite**
$2\text{KAl}_3\text{Si}_3\text{O}_{10}\text{(OH)}_2 \text{(s)} + 3\text{H}_2\text{O} \text{(l)}+ 2\text{H}^+ \text{(aq)} \leftrightharpoons 3\text{Al}_2\text{Si}_2\text{O}_5\text{(OH)}_4 \text{(s)} + 2 \text {K}^+ \text{(aq)}, \space K = 10^{6.741} \space \text{(reaction 3)}$

**microcline - muscovite**
$3\text{KAlSi}_3\text{O}_8 \text{(s)}+ 2\text{H}^+ \text{(aq)} + 12\text{H}_2\text{O} \text{(l)} \leftrightharpoons \text{KAl}_3\text{Si}_3\text{O}_{10}\text{(OH)}_2 \text{(s)} + 2 \text {K}^+ \text{(aq)} + 6\text{H}_{4}\text{SiO}_{4} \text{(aq)},\space K = 10^{-14.412} \space \text{(reaction 4)}$

In [ ]:
# students should assign the log K/H value for the equilibrium point 
# between kaolinite muscovite and microcline here
kaol_musc_micr_eq_point_KH <- ####
print(kaol_musc_micr_eq_point_KH)

In [ ]:
# students should assign the log H4SiO4 value for the equilibrium point 
# between kaolinite muscovite and microcline here
kaol_musc_micr_eq_point_H4SiO4 <- ####
print(kaol_musc_micr_eq_point_H4SiO4)

In [ ]:
# students should assign the log K/H value for where the microcline - muscovite 
# equilibrium line intersects the edge of the plotted area here
musc_micr_edge_KH <- ####
print(musc_micr_edge_KH)

In [ ]:
# students should assign the log H4SiO4 value for where the microcline - muscovite 
# equilibrium line intersects the edge of the plotted area here
musc_micr_edge_H4SiO4 <- ####
print(musc_micr_edge_H4SiO4)


[End answer 3]



In [ ]:
# the microcline - kaolinite line is solved for you
kaol_micr_edge_KH <- ####
print(kaol_micr_edge_KH)

Now, if you solved the equations and saved them to the appropriate variables, then running the following code chunk should generate a complete mineral stability diagram. It should look like the first figure you generated using the CHNOSZ package.



In [ ]:
plot_stability_segment5 <- 
    # start with the previous plot
    plot_stability_segment4 + 
    # muscovite - kaolinite equilibrium line
    geom_segment(aes(x = gibb_intersect_H4SiO4, y = gibb_intersect_KH, xend = kaol_musc_micr_eq_point_H4SiO4, yend = kaol_musc_micr_eq_point_KH)) + 
    #microcline - muscovite equilibrium line
    geom_segment(aes(x = kaol_musc_micr_eq_point_H4SiO4, y = kaol_musc_micr_eq_point_KH, xend = musc_micr_edge_H4SiO4, yend = musc_micr_edge_KH)) + 
    #microcline - kaolinite equilibrium line
    geom_segment(aes(x = kaol_musc_micr_eq_point_H4SiO4, y = kaol_musc_micr_eq_point_KH, xend = -2, yend = kaol_micr_edge_KH)) + 
    # add test labels
    annotate("text", y = 5.5, x= -4.6, label = "muscovite", size = 4)+
    annotate("text", y = 5.5, x= -3, label = "microcline", size = 4)+
    annotate("text", y = -2, x= -3, label = "kaolinite", size = 4)

# print plot
print(plot_stability_segment5)

# Reaction Path Modeling

Now that you have successfully derived the mineral stability diagram, we will trace changing water chemistry as microline weathers in the presence of soil water. This is a type of _reaction path_ modeling.

Our calculations are adapted from the model of Steinmann et al., (1994) (http://ccm.geoscienceworld.org/content/42/2/197), which is based the following assumptions:

1. The system is closed and the water/rock ratio is small. This is akin to having an essentially infinite amount of rock that is able to be weathered.
2. The secondary minerals are in equilibrium with ions in the solution even though the primary minerals are not in equilibrium.
3. Temperature and pressure are constant.
4. Aluminum is conserved in the reactions.
5. The solutions remain dilute such that all activity coefficients are close to unity.

## Microcline weathering to gibbsite

We are going to start with the single mineral microcline in typical acidic soil water, which has the following ion concentrations:

$\left[\text{H}_4\text{SiO}_4\right] = 10^{-6}$

$\left[\text{K}^{+}\right] = 10^{-6}$

$\text{pH} = 4$

Remember that $\text{pH} = -\log\left[\text{H}^+\right]$.

When you plot the initial water chemistry point in the chunk below, you should notice that it sits within the gibbsite field. This indicates that microcline is NOT stable in this water chemistry.

The next question is now which weathering reaction should we use for this initial step? Well, we know that we have microcline and that our water chemistry is in the gibbsite field. So, the weathering reaction we use first will be microcline to gibbsite. The following R code chunks will walk you though how this calculation is done.

**microcline to gibbsite**
$\text{KAlSi}_3\text{O}_8 + \text{H}^+ + 7\text{H}_2\text{O} \leftrightharpoons \text{Al(OH)}_3 + \text{K}^+ + 3\text{H}_4\text{SiO}_4  \space \text{(reaction 6)}$

## Question 4
Why did we not derive a phase diagram boundary line for the microcline to gibbsite reaction?

In the next chunk, we will assign variables for the initial chemical conditions. We then assign an arbitrary _reaction progress variable_, which is simply a placeholder to mark how far the reaction has progressed towards equilibrium. Note that this is not a kinetic model, and that the reaction progress does not accurately represent time. Then we assign variables for the coefficients of reaction 6, where positive values are assigned for the stoichiometry of products and negative values are for reactants. We then create a sequence of reaction steps, and calculate chemistry at each step from the stoichiometric coefficients. We then print out a data frame of ion concentrations and moles of mineral at each step, which you can have a look at to get a feel for the data.

## Question 5
Assign the initial proton concentration in moles per liter to the variable "mol_l_H_A" in the chunk below, then run it. Note: you can use scientific notation, i.e. $5*10^{3}$ can be written in the code as 5e3.

In [ ]:
#initial conditions are no secondary minerals (only primary mineral, microcline), and water chemistry of typical acidic soil
mol_gibbs_A <- 0 # moles gibbsite at t0
mol_l_H_A <- 1e-4 #### ENTER PROTON CONCENTRATION OF A TYPICAL ACIDIC SOIL, pH =4  # moles per liter H+ t0
mol_l_K_A <- 1e-6 # moles per liter K+ t0
mol_l_H4SiO4_A <- 1e-6 # moles per liter H4SiO4 t0
mol_kaol_A <- 0 # moles kaolinite t0
mol_musc_A <- 0 # moles muscovite t0

rxn_prog_var <- 1.882e-6 # reaction progress variable is proportional to the amount of reactant mineral dissolved and defines the extent of reaction. Selection of reaction progress step is arbitrary. Units are moles per kg of solution

#coefficients of change for weathering of microcline to gibbsite (reaction 6)
co_spar_1 <- -1
co_H_1 <- -1
co_H2O_1 <- -7
co_gibbs_1 <- 1
co_K_1 <- 1
co_H4SiO4_1 <- 3
co_kaol_1 <- 0
co_musc_1 <- 0

# make table of reaction steps
k_spar_to_gibbs <- 
    tibble(
        # steps
        steps = seq(0, 100, by = 0.001),
        # calculate geochemistry through reaction progress
        mol_l_H = mol_l_H_A + co_H_1 * steps * rxn_prog_var, 
        mol_l_K = mol_l_K_A + co_K_1 * steps * rxn_prog_var, 
        mol_l_H4SiO4 = mol_l_H4SiO4_A + co_H4SiO4_1 * steps * rxn_prog_var, 
        mol_gibbs = mol_gibbs_A + co_gibbs_1 * steps * rxn_prog_var, 
        mol_kaol = mol_kaol_A + co_kaol_1 * steps * rxn_prog_var, 
        mol_musc = mol_musc_A + co_musc_1 * steps * rxn_prog_var, 
        log_H4SiO4 = log10(mol_l_H4SiO4), 
        log_K_H_ratio = log10(mol_l_K/mol_l_H)
    ) %>% 
    filter(log_H4SiO4 <= gibb_intersect_H4SiO4)

# display first few lines of the reaction steps and chemistry data frame
head(k_spar_to_gibbs, 10)

[end answer 5]
Run this chunk to plot evolution of water chemistry during microcline weathering to gibbsite


In [ ]:
plot_AB <- 
    # start with previous plot
    plot_stability_segment5 %+%
    # add reaction progress line
    geom_point(
        data = k_spar_to_gibbs, 
        map = aes(x = log_H4SiO4, y = log_K_H_ratio, color = steps)) +
    # color scale
    scale_color_gradient(TeX("reaction progress ($\\xi$)"), low = "red", high = "green") +
    theme(legend.position = "bottom") +
    # annotate points A and B
    annotate("text", y = -2.4, x=-5.85, label = "A", size = 6) + 
    annotate("text", y = -1.3, x=-4.35, label = "B", size = 6) 

# print plot
options(repr.plot.width = 6, repr.plot.height = 5) # increase plot size for legend
print(plot_AB)

## Question 6
Look at the plot that was generated. Did silicic acid increase or decrease? What accounts for the change in silicic acid? What happened amount of gibbsite?

##  Microcline weathering: Gibbsite converted to Kaolinite

Now, we have some gibbsite in our system that was produced by K-feldspar weathering. However, if reaction 6 were to continue, the water chemistry would move out of the field of gibbsite stability. Yet, microcline is not in equilibrium with the water yet either. Thus, in order to approach stability with microcline, all gibbsite must be consumed (reaction 7).

$\text{KAlSi}_3\text{O}_8 + 2\text{Al(OH)}_3 + \text{H}^+  \leftrightharpoons 1.5\text{Al}_2\text{Si}_2\text{O}_5\text{(OH)}_4 + \text{K}^+ + 0.5\text{H}_2\text{O} \space \text{(reaction 7)}$

## Question 7
Input the stoichiometric coefficients of this reaction below and run the chunk to generate a data frame of water chemistries as reaction 7 progresses.

In [ ]:
mol_gibbs_B <- tail(k_spar_to_gibbs$mol_gibbs, 1) # moles gibbsite at point B
mol_l_H_B <-  tail(k_spar_to_gibbs$mol_l_H, 1) # moles per liter H+ at point B
mol_l_K_B <- tail(k_spar_to_gibbs$mol_l_K, 1) # moles per liter K+ at point B
mol_l_H4SiO4_B <- tail(k_spar_to_gibbs$mol_l_H4SiO4, 1) # moles per liter H4SiO4 at point B
mol_kaol_B <- tail(k_spar_to_gibbs$mol_kaol, 1) # moles kaolinite at point B
mol_musc_B <- tail(k_spar_to_gibbs$mol_musc, 1) # moles muscovite at point B

# USER INPUT : coefficients of change for weathering of k spar with conversion of gibbsite to kaolinite (reaction 7)
co_spar_2 <- ##
co_H_2 <- ##
co_H2O_2 <- ##
co_gibbs_2 <- ##
co_K_2 <- ##
co_H4SiO4_2 <- ##
co_kaol_2 <- ##
co_musc_2 <- ##

# make table of reaction steps
gibbs_to_kaol <- 
    tibble(
        # steps
        steps = seq(0, 100, by = 0.01),
        # calculate geochemistry through reaction progress
        mol_l_H = mol_l_H_B + co_H_2 * steps * rxn_prog_var,
        mol_l_K = mol_l_K_B + co_K_2 * steps * rxn_prog_var, 
        mol_l_H4SiO4 = mol_l_H4SiO4_B + co_H4SiO4_2 * steps * rxn_prog_var, 
        mol_gibbs = mol_gibbs_B + co_gibbs_2 * steps * rxn_prog_var, 
        mol_kaol = mol_kaol_B + co_kaol_2 * steps * rxn_prog_var, 
        mol_musc = mol_musc_B + co_musc_2 * steps * rxn_prog_var, 
        log_H4SiO4 = log10(mol_l_H4SiO4), 
        log_K_H_ratio = log10(mol_l_K/mol_l_H)
    ) %>%
    # filter out data points where there were calculated to be 
    # negative moles gibbsite which is impossible
    filter(mol_gibbs >= 0) %>%
    # move steps to start where previous reaction ended
    mutate(steps = round(max(k_spar_to_gibbs$steps), 2) + steps)

head(gibbs_to_kaol, 10)


Now plot the reaction path by running the chunk below


In [ ]:
plot_ABC <- 
    # start with previous plot
    plot_AB + 
    # continue reaction line
    geom_point(
        data = gibbs_to_kaol, 
        map = aes(x = log_H4SiO4, y = log_K_H_ratio, color = steps)) +
    # annotate point C
    annotate("text", y = -.3, x=-4.35, label = "C", size = 6) 

# print plot
print(plot_ABC)

Run the chunk below to plot accumulation of minerals with reaction progress.


In [ ]:
# new plot of mineral abundances
plot_minerals_ABC <- 
    bind_rows(k_spar_to_gibbs, gibbs_to_kaol) %>% 
    ggplot() + 
    geom_line(aes(x = steps, y = 1e6 * mol_musc, color = "µmoles muscovite"))+
    geom_line(aes(x = steps, y = 1e6 * mol_gibbs, color = "µmoles gibbsite"))+
    geom_line(aes(x = steps, y = 1e6 * mol_kaol, color = "µmoles kaolinite"))+
    scale_y_continuous(name = "µmoles mineral in system")+
    scale_x_continuous(name = TeX("reaction progress, $\\xi$"))+
    theme_bw() + theme(legend.position = "bottom") +
    labs(color = "")

# print plot
print(plot_minerals_ABC)

## Question 8
You have just plotted mineral abundance versus reaction progress. Why is there an inverse relationship between kaolinite and gibbsite after reaction progress = ~10? How does this relate to your phase diagram?

## Microcline weathering to kaolinite

Now, all of our gibbsite has been reacted into kaolinite. However, K-feldspar is still not in equilibrium with the water water chemistry, and further reaction will move through the kaolinite stability field. Run the chunk below to see this progress.

**microcline - kaolinite**
$2\text{KAlSi}_3\text{O}_8 + 2\text{H}^+ + 9\text{H}_2\text{O} \leftrightharpoons  \text{Al}_2\text{Si}_2\text{O}_5\text{(OH)}_4 + 2\text{K}^+ + 4\text{H}_{4}\text{SiO}_{4} \space \text{(reaction 5)}$

In [ ]:
mol_gibbs_C <- tail(gibbs_to_kaol$mol_gibbs, 1) # moles gibbsite at point C
mol_l_H_C <-  tail(gibbs_to_kaol$mol_l_H, 1) # moles per liter H+ at point C
mol_l_K_C <- tail(gibbs_to_kaol$mol_l_K, 1) # moles per liter K+ at point C
mol_l_H4SiO4_C <- tail(gibbs_to_kaol$mol_l_H4SiO4, 1) # moles per liter H4SiO4 at point C
mol_kaol_C <- tail(gibbs_to_kaol$mol_kaol, 1) # moles kaolinite  at point C
mol_musc_C <- tail(gibbs_to_kaol$mol_musc, 1) # moles muscovite  at point C

#coefficients of change for weathering of microcline to kaolinite (reaction 5)
co_spar_3 <- -2
co_H_3 <- -2
co_H2O_3 <- -9
co_gibbs_3 <- 0
co_K_3 <- 2
co_H4SiO4_3 <- 4
co_kaol_3 <- 1
co_musc_3 <- 0

# make table of reaction steps
kspar_to_kaol <-
    tibble(
        # steps
        steps = seq(0, 100, by = 0.0005),
        # calculate geochemistry through reaction progress
        mol_l_H = mol_l_H_C + co_H_3 * steps * rxn_prog_var, 
        mol_l_K = mol_l_K_C + co_K_3 * steps * rxn_prog_var, 
        mol_l_H4SiO4 = mol_l_H4SiO4_C + co_H4SiO4_3 * steps * rxn_prog_var, 
        mol_gibbs = mol_gibbs_C + co_gibbs_3 * steps * rxn_prog_var, 
        mol_kaol = mol_kaol_C + co_kaol_3 * steps * rxn_prog_var, 
        mol_musc = mol_musc_C + co_musc_3 * steps * rxn_prog_var, 
        log_H4SiO4 = log10(mol_l_H4SiO4), 
        log_K_H_ratio = log10(mol_l_K/mol_l_H)
    ) %>%
    # filter out rxn steps past equilibrium
    filter(log_K_H_ratio <= kaol_musc_micr_eq_point_KH) %>%
    # move steps to start where previous reaction ended
    mutate(steps = round(max(gibbs_to_kaol$steps), 2) + steps)

head(kspar_to_kaol, 10)


Now plot the reaction path by running the chunk below

In [ ]:
plot_ABCD <- 
    # start with previous plot
    plot_ABC + 
    # continue reaction line
    geom_point(
        data = kspar_to_kaol, 
        map = aes(x = log_H4SiO4, y = log_K_H_ratio, color = steps)) +
    # annotate point D
    annotate("text", y = 3.7, x=-3.85, label = "D", size = 6) 

# print plot
print(plot_ABCD)


Plot accumulation of minerals, points A - D


In [ ]:
plot_minerals_ABCD <-
    # start with previous plot
    plot_minerals_ABC %+%
    # replace plot data with new summaryof all steps
    bind_rows(k_spar_to_gibbs, gibbs_to_kaol, kspar_to_kaol) 

# print plot
print(plot_minerals_ABCD)


## Microcline weathering with conversion of kaolinite to muscovite, reaching equilibrium between all 3 minerals

Run the chunks below.

$\text{KAlSi}_3\text{O}_8 + \text{Al}_2\text{Si}_2\text{O}_5\text{(OH)}_4 + 3\text{H}_2\text{O} \leftrightharpoons \text{KAl}_3\text{Si}_3\text{O}_{10}\text{(OH)}_2 + 2\text{H}_4\text{SiO}_4 \space \text{(reaction 8)}$

Initial conditions


In [ ]:
mol_gibbs_D <- tail(kspar_to_kaol$mol_gibbs, 1) # moles gibbsite at point D
mol_l_H_D <-  tail(kspar_to_kaol$mol_l_H, 1) # moles per liter H+ at point D
mol_l_K_D <- tail(kspar_to_kaol$mol_l_K, 1) # moles per liter K+ at point D
mol_l_H4SiO4_D <- tail(kspar_to_kaol$mol_l_H4SiO4, 1) # moles per liter H4SiO4 at point D
mol_kaol_D <- tail(kspar_to_kaol$mol_kaol, 1) # moles kaolinite  at point D
mol_musc_D <- tail(kspar_to_kaol$mol_musc, 1) # moles muscovite  at point D

#coefficients of change for feldspar weathering with conversion of kaolinite to muscovite (reaction 4)
co_spar_4 <- -1
co_H_4 <- 0
co_H2O_4 <- -3
co_gibbs_4 <- 0
co_K_4 <- 0
co_H4SiO4_4 <- 2
co_kaol_4 <- -1
co_musc_4 <- 1

# make table of reaction steps
kspar_to_musc <-
    tibble(
        # steps
        steps = seq(0, 100, by = 0.001),
        # calculate geochemistry through reaction progress
        mol_l_H = mol_l_H_D + co_H_4 * steps * rxn_prog_var, 
        mol_l_K = mol_l_K_D + co_K_4 * steps * rxn_prog_var, 
        mol_l_H4SiO4 = mol_l_H4SiO4_D + co_H4SiO4_4 * steps * rxn_prog_var, 
        mol_gibbs = mol_gibbs_D + co_gibbs_4 * steps * rxn_prog_var, 
        mol_kaol = mol_kaol_D + co_kaol_4 * steps * rxn_prog_var, 
        mol_musc = mol_musc_D + co_musc_4 * steps * rxn_prog_var, 
        log_H4SiO4 = log10(mol_l_H4SiO4), 
        log_K_H_ratio = log10(mol_l_K/mol_l_H)
    ) %>%
    # filter out rxn steps past equilibrium
    filter(mol_kaol >= 0) %>%
    # move steps to start where previous reaction ended
    mutate(steps = round(max(kspar_to_kaol$steps), 2) + steps)

head(kspar_to_musc, 10)

Now plot the reaction path by running the chunk below

In [ ]:
plot_ABCDE <- 
    # start with previous plot
    plot_ABCD + 
    # continue reaction line
    geom_point(
        data = kspar_to_musc, 
        map = aes(x = log_H4SiO4, y = log_K_H_ratio, color = steps)) +
    # annotate point D
    annotate("text", y = 3.7, x=-3.4, label = "E", size = 6)

# print plot
print(plot_ABCDE)

## Question 9
Will microcline continue to dissolve after point E is reached? Why or why not?


Plot accumulation of minerals points A-E


In [ ]:
plot_minerals_ABCDE <-
    # start with previous plot
    plot_minerals_ABCD %+%
    # replace plot data with new summaryof all steps
    bind_rows(k_spar_to_gibbs, gibbs_to_kaol, kspar_to_kaol, kspar_to_musc) 

# print plot
print(plot_minerals_ABCDE)




## Animating the full reaction

Finally, let's animate the changes in water chemistry and mineral abundances together and watch the full reaction unfold!


In [ ]:
anim_data <- 
    # combine all steps
    bind_rows(
        filter(k_spar_to_gibbs, steps %% 0.5 == 0), 
        filter(gibbs_to_kaol, steps %% 0.5 == 0), 
        filter(kspar_to_kaol, steps %% 1 == 0), 
        filter(kspar_to_musc, steps %% 2 == 0)
    )

plot_reactions_ABCDE_anim <- 
    anim_data %>%
    plot_ly() %>%
    add_trace(x = ~log_H4SiO4,
            y = ~log_K_H_ratio,
            mode = "lines",
            #marker = list(color = "black"),
            line = list(color = "black")) %>%
    add_trace(x = ~log_H4SiO4,
            y = ~log_K_H_ratio,
            frame = ~steps,
            marker = list(color = "red", size = 10)) %>%
    layout(xaxis = list(title = "log[H4SiO4]", range = c(-6, -3)),
           yaxis = list(title = "log[K+/H+]", range = c(-3, 5)))

plot_minerals_rxns_ABCDE_anim <- 
    anim_data %>%
    rename("Gibbsite" = "mol_gibbs", "Kaolinite" = "mol_kaol", "Muscovite" = "mol_musc") %>%
    gather(key = "phase", value = "moles", Gibbsite, Kaolinite, Muscovite) %>%
    plot_ly() %>%
    add_trace(x = ~phase,
            y = ~moles*10^6,
            type = 'bar',
            frame = ~steps,
            color = ~phase) %>%
    layout(xaxis = list(title = "Reaction progress"),
           yaxis = list(title = "Micromoles mineral in system", range = c(0, 50)))

suppressMessages(suppressWarnings(
    subplot(plot_reactions_ABCDE_anim, plot_minerals_rxns_ABCDE_anim,
            nrows = 1,
            titleX = T,
            titleY = T,
            margin = 0.05) %>%
      layout(showlegend = FALSE, title = "Reaction Animation") %>%
      animation_opts(frame = 200, transition = 0, redraw = TRUE)
))